# KT CNN Replication

From: https://github.com/KeremTurgutlu/myCNN/blob/master/CUSTOM%20CNN%20WITH%20PYTORCH.ipynb

Attempting to replicate the notebook there, in order to get a starting template where the data is successfully loaded into a model for training.

From there I can decide on how to go forward, and how much/little to use the fastai API.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os
sys.path.insert(1, os.path.join('../'))

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
import torch.nn.functional as F

In [5]:
# train_idx = !ls data/train

In [6]:
# train_idx

['ice', 'ship', 'train.json']

In [ ]:
train_idx = !ls data/train
valid_idx = !ls data/valid

In [ ]:
train_idx = [int(i.split('.')[0]) for i in train_idx]
val_idx = [int(i.split('.')[0]) for i in val_idx]

In [13]:
# drr_idx = ['0000.yolo.yo', '0001.froyo.momo.yo', '0004.moyo.yo']
# drr_idx = [int(i.split('.')[0]) for i in drr_idx]; drr_idx

[0, 1, 4]

The idea of the above lines is to get a list of all the filenames in the train & val folders. Then create a list of integer converts of the 1st part of these fnames. This requires the 1st part of the names to be numbers themselves as exampled just above.

In [10]:
train_data = pd.read_json('data/train.json')
test_data = pd.read_json('data/test.json')

In [14]:
#create training data array
train_data_array = []
for i, row in train_data.iterrows():
    band_1 = row['band_1']
    band_2 = row['band_2']
    stacked = np.dstack([np.array(band_1).reshape(75, 75), np.array(band_2).reshape(75, 75),
                    np.array(band_1).reshape(75, 75) / np.array(band_2).reshape(75, 75),
                    (np.array(band_1).reshape(75, 75) + np.array(band_2).reshape(75, 75))/2])
    train_data_array.append(stacked)

train_data_array = np.array(train_data_array)

In [15]:
test_data_array = []
for i, row in test_data.iterrows():
    band_1 = row['band_1']
    band_2 = row['band_2']
    stacked = np.dstack([np.array(band_1).reshape(75, 75), np.array(band_2).reshape(75, 75),
                    np.array(band_1).reshape(75, 75) / np.array(band_2).reshape(75, 75),
                    (np.array(band_1).reshape(75, 75) + np.array(band_2).reshape(75, 75))/2])
    test_data_array.append(stacked)

test_data_array = np.array(test_data_array)

In [16]:
print(train_data_array.shape)
print(test_data_array.shape)

(1604, 75, 75, 4)
(8424, 75, 75, 4)


In [17]:
train_data_array = train_data_array.transpose((0, 3, 2, 1))
test_data_array = test_data_array.transpose((0, 3, 2, 1))

In [40]:
%cd ../..

/home/wnixalo/iceberg/temp


In [44]:
# %rm -r data/train/*.npy data/valid/*.npy

In [45]:
for idx in range(len(train_data_array)):
    np.save('data/train/'+'{:0=4d}'.format(idx), train_data_array[idx])

%cd data/train/

from glob import glob

g = glob('*.npy')
shuf = np.random.permutation(g)

for i in range(400):
    os.rename(shuf[i], '../valid/' + shuf[i])

%cd ../..

/home/wnixalo/iceberg/temp/data/train
/home/wnixalo/iceberg/temp


In [52]:
# %rmdir data/train/ice data/train/ship data/valid/ice data/valid/ship

In [53]:
train_idx = !ls data/train
valid_idx = !ls data/valid

In [54]:
train_idx = [int(i.split('.')[0]) for i in train_idx]
valid_idx = [int(i.split('.')[0]) for i in valid_idx]

In [57]:
valid_idx[:10], train_idx[:10]

([2, 3, 5, 8, 18, 19, 20, 26, 37, 39], [0, 1, 4, 6, 7, 9, 10, 11, 12, 13])

## CREATE PYTORCH DATASETS

In [59]:
# Creating FloatTensors from numpy arrays
train_data_tensor = torch.FloatTensor(train_data_array[train_idx, :, :, :])
valid_data_tensor = torch.FloatTensor(train_data_array[valid_idx, :, :, :])
test_data_tensor = torch.FloatTensor(test_data_array)

In [25]:
# blah_array = ['b','l','a','h','r','g']
blah_array = np.array([[i for i in range(10)] for r in range(10)])
print(blah_array)
print(drr_idx)
durr_array = blah_array[drr_idx]
print(durr_array)
# blah_array.shape

[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]
[0, 1, 4]
[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]


In [61]:
# Creating LongTensor from responses of train, valid, test
train_y_tensor = torch.LongTensor(np.array(train_data.is_iceberg.loc[train_idx].values).astype('int64'))
valid_y_tensor = torch.LongTensor(np.array(train_data.is_iceberg.loc[valid_idx].values).astype('int64'))
test_y_tensor = torch.LongTensor(np.zeros((test_data.shape[0], 1)).astype('int64'))

In [62]:
# Creating TensorDatasets
train_dataset = TensorDataset(data_tensor=train_data_tensor, target_tensor=train_y_tensor)
valid_dataset = TensorDataset(data_tensor=valid_data_tensor, target_tensor=valid_y_tensor)
test_dataset = TensorDataset(data_tensor=test_data_tensor, target_tensor=test_y_tensor)

In [63]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=15,
                                         shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(valid_dataset, batch_size=15,
                                         shuffle=True, num_workers=4)

In [64]:
dataloaders = {'train':trainloader, 'val':valloader}
datasizes = {'train':train_data_tensor.size(0), 'val':valid_data_tensor.size(0)}

## MODEL

In [65]:
def conv_calc(W, F, S, P): return (W - F + 2*P) / S + 1
def pool_calc(W, F, S): return (W - F) / S + 1

In [66]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=4,
                      kernel_size=4,
                      out_channels=64,
                      padding=1
            ), 
            nn.BatchNorm2d(64),
            nn.LeakyReLU()                   # try leak LeakyReLU later as well
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=64,
                      kernel_size=3,
                      out_channels=64,
                      padding=1
            ),                 
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),                   # try leak LeakyReLU later as well
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)
        )
        
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=64,
                      kernel_size=4,
                      out_channels=128,
                      padding=1
            ),              
            nn.BatchNorm2d(128),
            nn.LeakyReLU()                   # try leak LeakyReLU later as well
        )
        
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=128,
                      kernel_size=3,
                      out_channels=128,
                      padding=1
            ),             
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),                   # try leak LeakyReLU later as well
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)   
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=128,
                      kernel_size=3,
                      out_channels=256,
                      padding=1
            ),                   
            nn.BatchNorm2d(256),
            nn.LeakyReLU()                   # try leak LeakyReLU later as well
        )
        
        
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels=256,
                      kernel_size=3,
                      out_channels=256,
                      padding=1
            ),                   
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),                   # try leak LeakyReLU later as well
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)   
        )

        self.conv7 = nn.Sequential(
            nn.Conv2d(in_channels=256,
                      kernel_size=4,
                      out_channels=512,
                      padding=1
            ),            
            nn.BatchNorm2d(512),
            nn.LeakyReLU()                   # try leak LeakyReLU later as well
        )
        
        
        self.conv8 = nn.Sequential(
            nn.Conv2d(in_channels=512,
                      kernel_size=3,
                      out_channels=512,
                      padding=1
            ),
            nn.BatchNorm2d(512),           
            nn.LeakyReLU(),                   # try leak LeakyReLU later as well
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25)   
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(512*4*4, 2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5)
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5)
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(1024, 2),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5)
        )
    
    def forward(self, x):
        x = self.conv1(x) # output (64, 74, 74)
        x = self.conv2(x) # output (64, 37, 37)
        x = self.conv3(x) # output (128, 36, 36)
        x = self.conv4(x) # output (128, 18, 18)
        x = self.conv5(x) # output (256, 18, 18)
        x = self.conv6(x) # output (256, 9, 9)
        x = self.conv7(x) # output (512, 8, 8)
        x = self.conv8(x) # output (512, 4, 4)
        x = x.view(x.size(0), -1) # flatten the output 8192 nodes
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return F.log_softmax(x)

In [67]:
net = CNN()
optimizer = torch.optim.Adam(net.parameters(), lr=0.005)
loss_func = nn.CrossEntropyLoss()

In [68]:
net.parameters

<bound method Module.parameters of CNN (
  (conv1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.01)
  )
  (conv2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.01)
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Dropout (p = 0.25)
  )
  (conv3): Sequential (
    (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.01)
  )
  (conv4): Sequential (
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.01)
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Dropout (p = 0.25)
  )
 

In [69]:
n_cycles = 20

for epoch in range(n_cycles):
    for step, (x, y) in enumerate(trainloader):
        b_x = Variable(x)
        b_y = Variable(y)
        output = net(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch}')

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4


Process Process-24:
Process Process-23:
Process Process-21:
Process Process-22:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/wnixalo/miniconda3/envs/FAI/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/wnixalo/miniconda3/envs/FAI/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/wnixalo/miniconda3/envs/FAI/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wnixalo/miniconda3/envs/FAI/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/wnixalo/miniconda3/envs/FAI/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/wnixalo/miniconda3/envs/FAI/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwa

KeyboardInterrupt: 